# Autoencoding

In [ ]:
from tensorflow import keras

In [ ]:
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [ ]:
import matplotlib.pyplot as plt

n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 1.2, n_rows * 1.2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(x_train[index], cmap="gray", interpolation="nearest")
        plt.axis('off')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [ ]:
autoencoder = keras.models.Sequential()
autoencoder.add(keras.layers.Flatten(input_shape=x_train.shape[1:]))
autoencoder.add(keras.layers.Dense(128, activation="relu"))
autoencoder.add(keras.layers.Dense(50, activation="relu"))
autoencoder.add(keras.layers.Dense(128, activation="relu"))
autoencoder.add(keras.layers.Dense(x_train.shape[1]*x_train.shape[2], activation="relu"))
autoencoder.add(keras.layers.Reshape(x_train.shape[1:]))

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.compile(loss="mse",
              optimizer="adam",
              metrics=["mae"])

In [ ]:
H = autoencoder.fit(
	x_train, x_train,
	validation_data=(x_test, x_test),
	epochs=20,
	batch_size=64)

In [ ]:
N = range(0, 20)
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")

In [ ]:
N = range(0, 20)
plt.figure()
plt.plot(N, H.history["mae"], label="train_mae")
plt.plot(N, H.history["val_mae"], label="val_mae")
plt.title("Training MAE")
plt.xlabel("Epoch #")
plt.ylabel("MAE")
plt.legend(loc="lower left")

In [ ]:
reconstruct = autoencoder.predict(x_test)

In [ ]:
reconstruct.shape

In [ ]:
import numpy as np

n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 2, n_rows * 2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(np.concatenate((x_test[index], reconstruct[index]), axis=1), cmap="gray", interpolation="nearest")
        plt.axis('off')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [ ]:
autoencoder.layers

In [ ]:
autoencoder.summary()

In [ ]:
encoder = keras.models.Model(autoencoder.layers[0].input , autoencoder.layers[2].output)

In [ ]:
encoder.summary()

In [ ]:
encoded = encoder.predict(x_test)

In [ ]:
encoded.shape

# Denoising

In [ ]:
# sample noise from a random normal distribution centered at 0.5 (since
# our images lie in the range [0, 1]) and a standard deviation of 0.5

trainNoise = np.random.normal(loc=0.5, scale=0.5, size=x_train.shape)
testNoise = np.random.normal(loc=0.5, scale=0.5, size=x_test.shape)
x_trainNoisy = np.clip(x_train + trainNoise, 0, 1)
x_testNoisy = np.clip(x_test + testNoise, 0, 1)

In [ ]:
import numpy as np

n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 2, n_rows * 2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(np.concatenate((x_train[index], x_trainNoisy[index]), axis=1), cmap="gray", interpolation="nearest")
        plt.axis('off')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [ ]:
autoencoder = keras.models.Sequential()
autoencoder.add(keras.layers.Flatten(input_shape=x_train.shape[1:]))
autoencoder.add(keras.layers.Dense(128, activation="relu"))
autoencoder.add(keras.layers.Dense(50, activation="relu"))
autoencoder.add(keras.layers.Dense(128, activation="relu"))
autoencoder.add(keras.layers.Dense(x_train.shape[1]*x_train.shape[2], activation="relu"))
autoencoder.add(keras.layers.Reshape(x_train.shape[1:]))

autoencoder.compile(loss="mse",
              optimizer="adam",
              metrics=["mae"])

In [ ]:
H = autoencoder.fit(
	x_train, x_trainNoisy,
	validation_data=(x_test, x_testNoisy),
	epochs=20,
	batch_size=64)

In [ ]:
N = range(0, 20)
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")

N = range(0, 20)
plt.figure()
plt.plot(N, H.history["mae"], label="train_mae")
plt.plot(N, H.history["val_mae"], label="val_mae")
plt.title("Training MAE")
plt.xlabel("Epoch #")
plt.ylabel("MAE")
plt.legend(loc="lower left")

In [ ]:
denoised = autoencoder.predict(x_test)

n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 2, n_rows * 2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(np.concatenate((x_testNoisy[index], denoised[index]), axis=1), cmap="gray", interpolation="nearest")
        plt.axis('off')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

# Anomaly Detection

In [ ]:
autoencoder = keras.models.Sequential()
autoencoder.add(keras.layers.Flatten(input_shape=x_train.shape[1:]))
autoencoder.add(keras.layers.Dense(256, activation="relu"))
autoencoder.add(keras.layers.Dense(128, activation="relu"))
autoencoder.add(keras.layers.Dense(100, activation="relu"))
autoencoder.add(keras.layers.Dense(128, activation="relu"))
autoencoder.add(keras.layers.Dense(256, activation="relu"))
autoencoder.add(keras.layers.Dense(x_train.shape[1]*x_train.shape[2], activation="relu"))
autoencoder.add(keras.layers.Reshape(x_train.shape[1:]))

autoencoder.compile(loss="mse",
              optimizer="adam",
              metrics=["mae"])

H = autoencoder.fit(
	x_train, x_train,
	validation_data=(x_test, x_test),
	epochs=40,
	batch_size=512)

In [ ]:
N = range(0, 40)
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")

plt.figure()
plt.plot(N, H.history["mae"], label="train_mae")
plt.plot(N, H.history["val_mae"], label="val_mae")
plt.title("Training MAE")
plt.xlabel("Epoch #")
plt.ylabel("MAE")
plt.legend(loc="lower left")

In [ ]:
reconstruct = autoencoder.predict(x_test)

n_rows = 4
n_cols = 10
plt.figure(figsize=(n_cols * 2, n_rows * 2))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(np.concatenate((x_test[index], reconstruct[index]), axis=1), cmap="gray", interpolation="nearest")
        plt.axis('off')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

In [ ]:
np.abs(autoencoder.predict(x_train) - x_train).sum() / (len(x_train)*28*28)

In [ ]:
np.abs(autoencoder.predict(x_test) - x_test).sum() / (len(x_test)*28*28)

In [ ]:
plt.imshow(x_test[0], cmap='gray')

In [ ]:
plt.imshow(x_testNoisy[0], cmap='gray')

In [ ]:
anomaly = autoencoder.predict(x_testNoisy[0].reshape(1, 28, 28))
anomaly.shape

In [ ]:
plt.imshow(anomaly.squeeze(), cmap='gray')

In [ ]:
np.abs(autoencoder.predict(x_testNoisy[0].reshape(1, 28, 28)) - x_test[0]).sum() / (28*28)